In [1]:
using DifferentialEquations;
using Plots;
using Trapz

In [2]:
#parameters
T = 2
uref1 = 0.0
uref2 = 0.0
tau = 1
beta = 1;
L = 1;
m = 1;
g = 0;


epsilon = 0.2 #0.5 #0.2 #tau / sqrt(m * beta * (L^2) );
sf = epsilon^2;

samples = 5000
t = LinRange(0,T,samples);


#boundary conds
pos_sigma_start = 1
x_sigma_start = 0 #
mom_sigma_start = 1 # 

pos_sigma_end = 1.7
x_sigma_end = 0
mom_sigma_end = 1 

pos_mu_start = 0 
mom_mu_start = 0 

pos_mu_end = sqrt(2)
mom_mu_end = 0;

In [ ]:
#copied from existing code

#this is the function for the pde we are solving
function evolution!(du, u, p, t)
    v11, v12, v22, c11, c12, c22 = u
    epsilon, uref2 = p
    du[1] = 2*epsilon*uref2*v12 - 2*(v12^2) + 2*((c12*v22/c11)^2)
    du[2] = -((2*c12*(v22^2))/(c11)) - v12*(-1+2*v22) - epsilon*(v11-uref2*v22)
    du[3] = 2.0*(v22-v12*epsilon)
    du[4] = 2.0*epsilon*(c12)
    du[5] = -epsilon*(-c22+c11*uref2) + 2*c11*v12 - c12*(1-2*v22)
    du[6] = 2-2*c22 - 2*epsilon*c12*uref2 + 4*c12*v12 + ((4*(c12^2)*v22)/(c11))
end

#the boundary condition function 
function bc!(residual, u, p, t)
    residual[1] = u[1][4] - pos_sigma_start #the solution at the start
    residual[2] = u[1][5] - x_sigma_start
    residual[3] = u[1][6] - mom_sigma_start
    residual[4] = u[end][4] - pos_sigma_end #the solution at the end
    residual[5] = u[end][5] - x_sigma_end
    residual[6] = u[end][6] - mom_sigma_end
end

tspan = (0.0,T)

p = [epsilon,uref2]

#initial guess
u0 = [0.0,0.0,0.0,1.0,0.0,1.0]

bvp1 = BVProblem(evolution!, bc!, u0, tspan, p);

sol1 = solve(bvp1,MIRK4(),dt=0.0001)
    #


#plot(sol1)

In [ ]:
#functions
 
function v11(t) #
    return sol1(t)[1]
end

function v12(t) #
    return sol1(t)[2]
end

function v22(t) #
    return sol1(t)[3]
end

function c11(t) #position var
    return sol1(t)[4]
end

function c12(t) #cross var
    return sol1(t)[5]
end

function c22(t)  #mom var
    return sol1(t)[6]
end;


In [ ]:
plot(t,[v11.(t),v12.(t),v22.(t)])#,labels = ["Variance pos" "Cross-Variance" "Variance mom"])

In [ ]:
plot(t,[c11.(t),c12.(t),c22.(t)],labels = ["Variance pos" "Cross-Variance" "Variance mom"])

In [ ]:
#meansys
###DO NOT TOUCH

#this is the function for  pde we are solving
function evolution!(du, u, p, t)
    
    s1,s2,v1,v2 = u
    epsilon,uref2 = p
    
    du[1] = epsilon*s2
    
    du[2] = -epsilon*(uref1+s1*uref2) + 2*s1*v12(t) +2*v2-s2*(1-2*v22(t))
    
    du[3] = -2*v12(t)*v2 - epsilon*(-uref1*v12(t)-uref2*v2) - (2*(c12(t)^2)*s1*(v22(t)^2))/(c11(t)^2) + (2*c12(t)*s2*(v22(t)^2))/(c11(t))
    
    du[4] = (2*c12(t)*s1*(v22(t)^2))/(c11(t)) - 2*s2*(v22(t)^2) - v2*(-1+2*v22(t)) - epsilon*(v1-uref1*v22(t))

end

#the boundary condition function 
function bc!(residual, u, p, t)
    residual[1] = u[1][1] - pos_mu_start #the solution at the start
    residual[2] = u[1][2] - mom_mu_start
    residual[3] = u[end][1] - pos_mu_end #the solution at the end
    residual[4] = u[end][2] - mom_mu_end
end

tspan = (0.0,T)

#initial guess
u0 = [0.0, 0.0, 0.1, 0.2]

bvp2 = BVProblem(evolution!, bc!, u0, tspan, p);

#testing things out
meansyssol2 = solve(bvp2, MIRK4(), dt = 0.0001);
#sol1 = solve(bvp1, Shooting(Vern7()), dt = 0.0001);

#gets close!
#meansyssol = solve(bvp2, MIRK4(), dt = 0.01);



In [ ]:
#functions for easier naming
 
function s1(t) #position mean
    return meansyssol2(t)[1]
end

function s2(t) #momentum mean
    return meansyssol2(t)[2]
end

function v1(t)
    return meansyssol2(t)[3]
end

function v2(t)
    return meansyssol2(t)[4]
end;


In [ ]:

plot(t,[v1.(t),v2.(t)])#,v1.(t),v2.(t)])

In [ ]:
plot(t,[s1.(t),s2.(t)])#,v1.(t),v2.(t)])

In [ ]:
#non-perturbative solution

plt = plot(t, [c11.(t),c12.(t),c22.(t),s1.(t),s2.(t)],layout =5, 
    titles = ["Position var" "cross var" "mom var" "position mean" "mom mean"],
    titlefontsize = 12,
    legend =false)

plot(plt)

In [ ]:
##perturbative expansion version1 
#sf=1#epsilon^2
p = [T,uref1, uref2];

#this is the function for the pde we are solving
function evolution!(du, u, p, t)
    nu_t, mu_t, val_1, val_2  = u
    T, uref1, uref2 = p
    
    #positin var
    du[1] = (-sf/((exp(T)*T*(2 + cosh(T)))))*(-3 + 3*exp(2*T) - T - 4*exp(T)*T - exp(2*T)*T + (3 +exp(2*T)*(-3 + T) + T + 4*exp(T)*T)*nu_t*(uref2 + 2*val_2)) 
    
    #position mean
    du[2] = sf*((-1/(T*cosh(T/2)))*((1/(2*(2 + cosh(T))*nu_t))*(-(mu_t*((5*T*cosh(T/2) + T*cosh((3*T)/2) - 4*sinh((3*T)/2))*nu_t*(uref2 + 2*val_2) + 2*cosh(T/2)*(-2*T*(2 + cosh(T)) + 6*sinh(T) + T*(2 + cosh(T))*du[1]/sf)))) - 2*sinh(T/2)*(uref1+ 2*val_1)) - (uref1+ 2*val_1))
    
    #=(-sf/(nu_t*(2*(1 + exp(T))*T*(2 + cosh(T)))))*((2 + exp(T)*(-2 + T) + T)*(2 + cosh(T))*uref1*nu_t - 2*(2 + exp(T)*(-2 + T) + T)*(2 + cosh(T))*uref2*mu_t*nu_t + 
                  2*(2 + exp(T)*(-2 + T) + T)*(2 + cosh(T))*nu_t*val_1 - 
                  4*(2 + exp(T)*(-2 + T) + T)*(2 + cosh(T))*mu_t*nu_t*val_2 + 
                  2*exp(-T)*mu_t*(1 - nu_t*(uref2 + 2*val_2)) + 
                  6*mu_t*(-1 + nu_t*(uref2 + 2*val_2)) + 
                  2*mu_t*(8 + 3*exp(T) + 4*T + 2*(2 + T)*cosh(T) - 
                  3*exp(T)*nu_t*(uref2 + 2*val_2))- 2*exp(T)*mu_t*(8 + exp(T)-4*T-2*(-2 + T)*cosh(T) - 
                  exp(T)*nu_t*(uref2 + 2*val_2)) - 
                  2*(1 + exp(T))*T*(2 + cosh(T))*mu_t*(du[1]/sf)) 
    
    
    #=(sf/(nu_t*2*T*(2+cosh(T))))*((-mu_t*(8*T+4*T*cosh(T)- 12*sinh(T)-
                ((5*T*cosh(T/2) + T*cosh(3*T/2) - 4*sinh(3*T/2))/(cosh(T/2)))*nu_t*(uref2+2*val_2)-2*T*(2+cosh(T))*du[1]/sf)) 
                -nu_t*(uref1*(2*T+T*cosh(T) +(8*(sinh(T/2)^4)/sinh(T)) -3*sinh(T))+
                (4*T+2*T*cosh(T)+(16*(sinh(T/2)^4)/sinh(T))-6*sinh(T))*val_1)) =#
 
    =#
    
    #val_1 
    du[3] = (-1/((((T +exp(T)*T)/sf))*(2*(1 + 4*exp(T) +exp(2*T))*(nu_t^2))))*(-((-1 +exp(T))*mu_t*(-3*(1 +exp(T))^2 + (-1 +exp(T))^2*(nu_t^2)*(uref2 + 2*val_2)^2)) + (2*(1 + 4*exp(T) +exp(2*T))*(nu_t^2)*(-uref1*uref2 +exp(T)*uref1*uref2 - (2 +exp(T)*(-2 + T) + T)*uref1*val_2 - (2 +exp(T)*(-2 + T) + T)*val_1*(uref2 + 2*val_2))))
    
    #-sf/((T+exp(T)*T)*2*(1+4*exp(T)+exp(2*T))*nu_t^2))*(-((-1 +exp(T))*mu_t*(-3*(1 + exp(T))^2 + (-1 +exp(T))^2*nu_t^2*(uref2+ 2*val_2)^2)) + 2*(1+4*exp(T) +exp(2*T))*nu_t^2*(-uref1*uref2 +exp(T)*uref1*uref2 - (2 +exp(T)*(-2 + T) + T)*uref1*val_2 - (2 +exp(T)*(-2 + T) + T)*val_1*(uref2+2*val_2))) 
    
    #(-1/((2*(1 + 4*exp(T) +exp(2*T))*nu_t^2*(T/sf + (exp(T)*T)/sf))))*(-((-1 +exp(T))*mu_t*(-3*(1 +exp(T))^2 + (-1 +exp(T))^2*nu_t^2*(uref2 + 2*val_2)^2)) +2*(1 + 4*exp(T) +exp(2*T))*nu_t^2*(-uref1*uref2 +exp(T)*uref1*uref2 - (2 +exp(T)*(-2 + T) + T)*uref1*val_2 - (2 + exp(T)*(-2 + T) + T)*val_1*(uref2 + 2*val_2)))
    
    #sf*((-1 / ((nu_t^2)*T*(2 + 10*exp(T) + 10*exp(2*T) +2*exp(3*T)))) *(-2* (-1 + exp(T))* mu_t * (-3*((1 + exp(T))^2) + ((-1 +exp(T))^2)*(nu_t^2)*((uref2 + 2*val_2)^2)) + (nu_t^2)* (-(uref1*uref2) - 9*exp(T)*(uref1*uref2) + 9*exp(2*T)*(uref1*uref2) +  exp(3*T)*(uref1*uref2) - 2*(1 + exp(3*T)*(-1 + T) + T + exp(2*T)*(-9 + 5*T) + exp(T)*(9 + 5*T)) * uref1* val_2 - 2*(1 + exp(3*T)*(-1 + T) + T + exp(2*T)*(-9 + 5*T) + exp(T)*(9 + 5*T))*val_1*(uref2 + 2*val_2))))
    
    #
    #
    #
    
    #val_2
    du[4] = (sf/(4*exp(T)*T*(2 + cosh(T))))*((-1/nu_t^2)*(3 - 3*exp(2*T)) - (3*(-1 +exp(2*T))*uref2^2 - 8*exp(T)*uref2*(2*T + T*cosh(T) - 3*sinh(T))*val_2 -8*exp(T)*(2*T + T*cosh(T) - 3*sinh(T))*val_2^2))
    
 end

#the boundary condition function 
function bc!(residual, u, p, t)
    residual[1] = u[1][1] - 1.0 #nu(0) at the start
    residual[2] = u[1][2] - 0.0 #mu(0) solution at the start
    residual[3] = u[end][1] - 1.7 #nu(T) solution at the end
    residual[4] = u[end][2] - sqrt(2) #mu(T) solution at the start
end

tspan = (0.0,T)

#initial guess
u0 = [1.0,0.0,-100.0,-1.0]

bvp1 = BVProblem(evolution!, bc!, u0, tspan, p);

perturb_sol2 = solve(bvp1,Shooting(Feagin14()), dt = 0.0001);#;
# MIRK4(), dt = 0.01)
    #


In [ ]:
#epsilon =1
function nu(t) #positin var
    return perturb_sol2(t)[1]
end

function mu(t) 
    return perturb_sol2(t)[2]
end

function val_1(t) #value
    return perturb_sol2(t)[3]
end

function val_2(t) #value
    return perturb_sol2(t)[4] 
end

function trig_exp(t)
    return (1/cosh(T/2))*sinh(t/2)*sinh((t-T)/2)
end


In [ ]:
#matches


plot(t,[nu.(t),mu.(t),val_1.(t),val_2.(t)],layout = 4)


In [ ]:
##matches!!! (do not touch!)

#varmom1
function mom_var(t)
    return (sinh(t/2)^2*(cosh(t)+sinh(t))*(-3+cosh(t)+3*cosh(t-T)-cosh(T)-sinh(t)+sinh(t-T)+sinh(T))^2*(mu(t)^2+nu(t))*(-1+nu(t)*(uref2+2*val_2(t)))^2)/(2*(2+cosh(T))^2*nu(t)^2)+1/2*(uref1^2*(1-cosh(t)+sinh(t)*tanh(T/2))^2+((1-cosh(t)+sinh(t)*tanh(T/2))^2*mu(t)^2)/nu(t)^2+(2*uref1*(1-cosh(t)+sinh(t)*tanh(T/2))^2*mu(t))/nu(t)+4*uref1*(1-cosh(t)+sinh(t)*tanh(T/2))^2*val_1(t)+(4*(1-cosh(t)+sinh(t)*tanh(T/2))^2*mu(t)*val_1(t))/nu(t)+4*(1-cosh(t)+sinh(t)*tanh(T/2))^2*val_1(t)^2+(2*exp(-2*t-T)*(-1+exp(t))^2*(exp(t)-exp(T))^2*(exp(t)+exp(T)+exp(t+T))*mu(t)^2*(-1+nu(t)*(uref2+2*val_2(t))))/((1+exp(T))^2*(2+cosh(T))*nu(t)^2)+(exp(-2*t)*(-1+exp(t))^2*(exp(t)-exp(T))^2*(-1+nu(t)*(uref2+2*val_2(t))))/((1+4*exp(T)+exp(2*T))*nu(t))+(2*exp(-2*t-T)*(-1+exp(t))^2*(exp(t)-exp(T))^2*(exp(t)+exp(T)+exp(t+T))*uref1*mu(t)*(-1+nu(t)*(uref2+2*val_2(t))))/((1+exp(T))^2*(2+cosh(T))*nu(t))+(4*exp(-2*t-T)*(-1+exp(t))^2*(exp(t)-exp(T))^2*(exp(t)+exp(T)+exp(t+T))*mu(t)*val_1(t)*(-1+nu(t)*(uref2+2*val_2(t))))/((1+exp(T))^2*(2+cosh(T))*nu(t))+(exp(-2*t+2*T)*(-1+exp(t))^2*(1+exp(t)-exp(2*t-2*T)*(1+exp(T)))^2*mu(t)^2*(-1+nu(t)*(uref2+2*val_2(t)))^2)/((1+exp(T))^2*(2+cosh(T))^2*nu(t)^2))-(4/cosh(T/2)*sinh(t/2)^2*(cosh(t)+sinh(t))*sinh((t-T)/2)^2*(4*cosh(T/2)*sinh(t/2)-2*(cosh(t/2)+sinh(t/2))*sinh(T/2))*mu(t)*(-1+nu(t)*(uref2+2*val_2(t)))*((2+cosh(T))*(cosh(t/2)-sinh(t/2))*nu(t)*(uref1+2*val_1(t))+mu(t)*(-2*sinh(t/2)-2*cosh(T)*sinh(t/2)+cosh(t/2)*sinh(T)+sinh(t/2)*sinh(T)+(cosh(T/2)-sinh(T/2))*(2*cosh((t-T)/2)+cosh((t+T)/2)+sinh((t+T)/2))*nu(t)*(uref2+2*val_2(t)))))/((2+cosh(T))^2*nu(t)^2)
    #(exp(-2*(t + T))*(-1 +exp(t))^2*(4*(-exp(t) + 2*exp(2*t)+exp(T) +exp(2*T) - 3*exp(t + T))^2*(mu(t)^2 + nu(t))*(-1 + nu(t)*(uref2+2*val_2(t)))^2+1/(1 +exp(T))*4*(exp(t)-exp(T))^2*(1-2*exp(t)+exp(T))*mu(t)*(-1+nu(t)*(uref2+2*val_2(t)))*(-2*(-1 +2*exp(t)-exp(T))*(1 +exp(T))*mu(t)+(1+4*exp(T)+exp(2*T))*nu(t)*(uref1+2*val_1(t))+4*(exp(t)+exp(T)+exp(t + T))*mu(t)*nu(t)*(uref2+2*val_2(t)))+1/(1+exp(T))^2*(exp(t)-exp(T))^2*(4*mu(t)^2*((-1+2*exp(t)-exp(T))*(1+exp(T))-2*(exp(t)+exp(T)+exp(t+T))*nu(t)*(uref2+2*val_2(t)))^2+4*(1+4*exp(T)+exp(2*T))*mu(t)*nu(t)*(uref1+2*val_1(t))*(-((-1+2exp(t)-exp(T))*(1 +exp(T)))+2*(exp(t)+exp(T)+exp(t + T))*nu(t)*(uref2+2*val_2(t)))+(1+4*exp(T)+exp(2*T))*nu(t)*(-4*(1 +exp(T))^2+nu(t)*(uref1^2+4*exp(T)*uref1^2+exp(2*T)*uref1^2+4*uref2+8*exp(T)*uref2+4*exp(2*T)*uref2+4*(1 + 4*exp(T)+exp(2*T))*uref1*val_1(t)+4*(1+4*exp(T)+exp(2*T))*val_1(t)^2+8*(1 +exp(T))^2*val_2(t))))))/(32*(2 + cosh(T))^2*nu(t)^2)
end

plot(t,mom_var.(t))

In [ ]:

#meanmom[1]
function mom_mean(t) #f^11_0,t_2
      return epsilon*(-(1/((2 + cosh(T))*nu(t)))*
    sinh(t/2)*(cosh(t/2) + sinh(t/2))*(-3 + cosh(t) + 3*cosh(t - T) - 
       cosh(T) - sinh(t) + sinh(t - T) + sinh(T))*mu(t)*(-1 + nu(t)*(uref2+ 2*val_2(t))) + (
   1/nu(t))*((-1 + cosh(t) - sinh(t)*tanh(T/2))*mu(t) + 
     uref1*(-1 + cosh(t) - sinh(t)*tanh(T/2))*nu(t) + 
     2*(-1 + cosh(t) - sinh(t)*tanh(T/2))*nu(t)*val_1(t) - (
    exp(-t + T)*(1 - exp(t))*(1 + exp(t) - exp(2*t - 2*T)*(1 + exp(T)))*mu(t)*
        (1 - nu(t)*(uref2+2*val_2(t))))/((1+exp(T))*(2+cosh(T))))) 
    
    #
end;


plot(t,[mom_mean.(t)])


In [ ]:
##matches
function cross_cor(t)
    return -((exp(-t-T)*(-1+exp(t))*(-exp(t)+2*exp(2*t)+exp(T)+exp(2*T)-3*exp(t+T))*(mu(t)^2+nu(t))*(-1+nu(t)*(uref2+2*val_2(t))))/(2*(2+cosh(T))*nu(t)))+(exp(-t-T)*(-1+exp(t))*(exp(t)-exp(T))*mu(t)*((1+4*exp(T)+exp(2*T))*nu(t)*(uref1+2*val_1(t))+mu(t)*(-((-1+2*exp(t)-exp(T))*(1+exp(T)))+2*(exp(t)+exp(T)+exp(t+T))*nu(t)*(uref2+2*val_2(t)))))/(2*(1+exp(T))*(2+cosh(T))*nu(t))
end

plot(cross_cor.(t))

In [ ]:
##matches!
##MEANPOSCOR1

function mean_pos_cor(t)
    return (epsilon^2)*(-((exp(-t-T)*(exp(t+2*T)*(3*t-T)-3*exp(2*t)*T+exp(3*t)*T+exp(T)*T+exp(2*T)*T 
        + 2*exp(t+T)*T-3*exp(2*t +T)*T+exp(t)*(-3*t+2*T))*mu(t)*(-1+nu(t)*(uref2+2*val_2(t))))/(2*T*(2+cosh(T))*nu(t)))
        +1/2*(-(1/(2+cosh(T)))*exp(-t - T)*(exp(2*T)*(1+exp(t)*(-1+t))+exp(t)*(2 + exp(t)*(-3+exp(t))+ t)+ 
        exp(T)*(1+exp(t)*(2 - 3*exp(t)+4*t)))*uref1+(exp(-T)*t*(3+exp(2*T)*(-3+T)+T+4*exp(T)*T)*uref1)/(T*(2 +cosh(T)))
        -(exp(-t-T)*(exp(2*T)*(1 + exp(t)*(-1 + t))+exp(t)*(2 + exp(t)*(-3 +exp(t))+t)+exp(T)*(1+exp(t)*(2-3*exp(t)+ 4*t)))*mu(t))/((2 + cosh(T))*nu(t)) 
        +(exp(-T)*t*(3 + exp(2*T)*(-3+T)+ T +4*exp(T)*T)*mu(t))/(T*(2+cosh(T))*nu(t))-(1/(2+ cosh(T)))*2*exp(-t - T)*(exp(2*T)*(1 +exp(t)*(-1 + t))+ exp(t)*(2 + exp(t)*(-3+exp(t))+t)+
            exp(T)*(1+exp(t)*(2 -3*exp(t)+4*t)))*val_1(t)+ (2*exp(-T)*t*(3+ exp(2*T)*(-3 + T)+T+4*exp(T)*T)*val_1(t))/(T*(2+cosh(T)))+(2*exp(-t)*(-1 + exp(t))^2*(exp(t)-2*exp(2*T)+exp(t + T))*(uref1+2*val_1(t)
                    + mu(t)*(uref2+2*val_2(t))))/(1+5*exp(T)+5*exp(2*T)+ exp(3*T))+ (2*(-1+exp(T))^3*t*(uref1+2*val_1(t)+mu(t)*(uref2+ 2*val_2(t))))/((1 + 5*exp(T)+5*exp(2*T)+exp(3*T))*T)))

end

plot(mean_pos_cor.(t))

In [ ]:
##matches!!!

##MEANPOSCOR1

function var_pos_cor(t)
    return (epsilon^2)*(-((exp(-t-T)*(exp(t+2*T)*(3*t-T)-3*exp(2*t)*T+exp(3*t)*T+exp(T)*T+exp(2*T)*T+2*exp(t+T)*T-3*exp(2*t+T)*T+exp(t)*(-3*t+2*T))*(mu(t)^2+nu(t))*(-1+nu(t)*(uref2+2*val_2(t))))/(T*(2+cosh(T))*nu(t)))+mu(t)*(-(1/(2+cosh(T)))*exp(-t-T)*(exp(2*T)*(1+exp(t)*(-1+t))+exp(t)*(2+exp(t)*(-3+exp(t))+t)+exp(T)*(1+exp(t)*(2-3*exp(t)+4*t)))*uref1+(exp(-T)*t*(3+exp(2*T)*(-3+T)+T+4*exp(T)*T)*uref1)/(T*(2+cosh(T)))-(exp(-t-T)*(exp(2*T)*(1+exp(t)*(-1+t))+exp(t)*(2+exp(t)*(-3+exp(t))+t)+exp(T)*(1+exp(t)*(2-3*exp(t)+4*t)))*mu(t))/((2+cosh(T))*nu(t))+(exp(-T)*t*(3+exp(2*T)*(-3+T)+T+4*exp(T)*T)*mu(t))/(T*(2+cosh(T))*nu(t))-(1/(2+cosh(T)))*2*exp(-t-T)*(exp(2*T)*(1+exp(t)*(-1+t))+exp(t)*(2+exp(t)*(-3+exp(t))+t)+exp(T)*(1+exp(t)*(2-3*exp(t)+4*t)))*val_1(t)+(2*exp(-T)*t*(3+exp(2*T)*(-3+T)+T+4*exp(T)*T)*val_1(t))/(T*(2+cosh(T)))+(2*exp(-t)*(-1+exp(t))^2*(exp(t)-2*exp(2*T)+exp(t+T))*(uref1+2*val_1(t)+mu(t)*(uref2+2*val_2(t))))/(1+5*exp(T)+5*exp(2*T)+exp(3*T))+(2*(-1+exp(T))^3*t*(uref1+2*val_1(t)+mu(t)*(uref2+2*val_2(t))))/((1+5*exp(T)+5*exp(2*T)+exp(3*T))*T)))
end

plot(var_pos_cor.(t))

In [ ]:
###MATCHES!

#position mean
p1 = plot(t, [mu.(t).+ mean_pos_cor.(t),s1.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    title ="Position mean",
    titlefontsize = 12,
    legend =true)

plot(p1)

In [ ]:
#momentum mean

p2 =plot(t, [mom_mean.(t),s2.(t)],
    labels = ["eps = $epsilon" "non-perturbative"],
    title="mom mean",
    
    titlefontsize = 12,
    legend =true)

plot(p2)

In [ ]:
#pos VAR

function pos_var_plot(t)
    return nu(t) + var_pos_cor(t) - 2*mu(t)*mean_pos_cor(t)
end

p3 = plot(t, [pos_var_plot.(t),c11.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    titlefontsize = 12,
    title = "Position Variance",
    legend =true)

plot(p3)

In [ ]:
#CROSS VAR

function cross_cor_plot(t)
    return epsilon*cross_cor(t)-mom_mean(t)*mu(t)
end

#need to add corrections 
p4 = plot(t, [cross_cor_plot.(t),c12.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    title="cross-variance",
    titlefontsize = 12,
    legend =true)


plot(p4)

In [ ]:
#MOM VAR
#need to fix mom-mean

function mom_var_plot(t)
    
    return 1 + 2*sf*mom_var(t) - ((mom_mean(t))^2)
end

p5 = plot(t, [mom_var_plot.(t),c22.(t)],
    labels = ["eps = $epsilon" "non-perturbative"],
    titlefontsize = 12,
    title="momentum variance",
    legend =true)

plot(p5)

In [ ]:
###MATCHES!
p6 = plot([zeros(1),zeros(2)],alpha=[1,1],labels = ["eps = $epsilon" "non-perturbative"],axis=([], false),leg_position = "left")

#position mean
plot(p1,p2,p3,p4,p5, p6,legend = [false false false false false true],layout = 6)


In [ ]:
using DataFrames;
using CSV;


file_name = "kl_gauss_T2_v4.csv"

# Define the header as an array of strings
row = ["t" "p_mom_var" "p_pos_var" "p_cc" "p_mom_mean" "p_pos_mean" "mom_var" "pos_var" "cc" "mom_mean" "pos_mean"]
header = DataFrame(row,vec(row))


t_vals = vec(t)
p_mom_var = vec(mom_var_plot.(t))
p_pos_var = vec(pos_var_plot.(t))
p_cc = vec(cross_cor_plot.(t))
p_mom_mean = vec(mom_mean.(t))
p_pos_mean = vec(mu.(t).+ mean_pos_cor.(t))
mom_var_vec = vec(c22.(t))
pos_var_vec = vec(c11.(t))
cc_vec = vec(c12.(t))
mom_mean_vec = vec(s2.(t))
pos_mean_vec = vec(s1.(t))


df = DataFrame([t_vals,p_mom_var,p_pos_var,p_cc,p_mom_mean,p_pos_mean,mom_var_vec,pos_var_vec,cc_vec,mom_mean_vec,pos_mean_vec],vec(row))


# Write the header to a new CSV file
CSV.write(file_name, header;header =false)

CSV.write(file_name, df,append =true)
